# Семинар 6. Конфигурирование MongoDB

1. Установить на локальный компьютер MongoDB.

2. Запустить сервер, произвести тестовое подключение

3. Из реляционой БД MySQL "monitor_vuz_2014" прочитать таблицу "region" и импортировать ее в виде документа в коллекцию MongoDB.

4. Построить репликацию хранилища из одного сервера и трех реплик. Выполнить п.3 на реплицированном хранилище.

5. Смоделировать выход из строя третьей реплики и главного сервера. 

In [3]:
import pymysql
sqlcon = pymysql.connect(host='localhost',user='juna',password='2',db="monitor_vuz")
curs = sqlcon.cursor()

In [4]:
curs.execute("describe region")
descr = curs.fetchall()
collsnames = [d[0] for d in descr]
print(collsnames)

['id', 'name', 'okrug_id']


In [5]:
print(descr)

(('id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('name', 'varchar(200)', 'YES', '', None, ''), ('okrug_id', 'int', 'NO', 'MUL', None, ''))


In [6]:
curs.execute('select * from region;')
data = curs.fetchall()
for d in data:
    print(d)

(1, 'Амурская область', 1)
(2, 'Еврейская автономная область', 1)
(3, 'Камчатский край', 1)
(4, 'Магаданская область', 1)
(5, 'Приморский край', 1)
(6, 'Республика Саха (Якутия)', 1)
(7, 'Сахалинская область', 1)
(8, 'Хабаровский край', 1)
(9, 'Чукотский автономный округ', 1)
(10, 'Кировская область', 2)
(11, 'Нижегородская область', 2)
(12, 'Оренбургская область', 2)
(13, 'Пензенская область', 2)
(14, 'Пермский край', 2)
(15, 'Республика Башкортостан', 2)
(16, 'Республика Марий Эл', 2)
(17, 'Республика Мордовия', 2)
(18, 'Республика Татарстан', 2)
(19, 'Самарская область', 2)
(20, 'Саратовская область', 2)
(21, 'Удмуртская Республика', 2)
(22, 'Ульяновская область', 2)
(23, 'Чувашская Республика', 2)
(24, 'Архангельская область', 3)
(25, 'Вологодская область', 3)
(26, 'Калининградская область', 3)
(27, 'Ленинградская область', 3)
(28, 'Мурманская область', 3)
(29, 'Ненецкий автономный округ', 3)
(30, 'Новгородская область', 3)
(31, 'Псковская область', 3)
(32, 'Республика Карелия', 3)

In [12]:
import pymongo
mcon = pymongo.MongoClient("localhost",27017)
print(mcon)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [13]:
dbname='test'
db = mcon[dbname]
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')


In [14]:
collectionname = "region"
collection = db[collectionname]
print(collection)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'region')


In [15]:
for d in data:
    collection.insert_one({k:v for (k,v) in zip(collsnames,d)})

4. Произвести замену в документе region атрибута okrug_id с номера на название из реляционной таблицы okrug.

In [16]:
curs.execute('describe okrug;')
res = curs.fetchall()
regcolls = [d[0] for d in res]
print(regcolls)

['id', 'name']


In [17]:
curs.execute('select * from okrug;')
regdata = curs.fetchall()
print(regdata[0])

(1, 'Дальневосточный федеральный округ')


In [20]:
for d in regdata:
    collection.update_many({"okrug_id":d[0]},{"$set":{"okrug_id":d[1]}})

In [19]:
regdata

((1, 'Дальневосточный федеральный округ'),
 (2, 'Приволжский федеральный округ'),
 (3, 'Северо-Западный федеральный округ'),
 (4, 'Северо-Кавказский федеральный округ'),
 (5, 'Сибирский федеральный округ'),
 (6, 'Уральский федеральный округ'),
 (7, 'Центральный федеральный округ'),
 (8, 'Южный федеральный округ'))